In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
spark = SparkSession. \
builder. \
appName('UNIR'). \
config("spark.executor.memory", "1g"). \
config("spark.driver.memory", "2g"). \
config('spark.dirver.maxResultSize', '1g'). \
config('spark.jars.packages','org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2'). \
getOrCreate()

22/07/25 03:55:14 WARN Utils: Your hostname, vagrant resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface eth0)
22/07/25 03:55:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/miniconda/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vagrant/.ivy2/cache
The jars for the packages stored in: /home/vagrant/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3658d1c4-a067-47ae-9e84-86aa7dfac112;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 267ms :: artifacts dl 4ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from centra

In [27]:
def promediarValores(df):
   df.createOrReplaceTempView("resultadoMedio")
   promedios = spark.sql("""SELECT tipo, AVG(total) AS promedio FROM resultadoMedio GROUP BY tipo ORDER BY promedio DESC""")
   return promedios

In [28]:
tiposStreamingDF = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "127.0.0.1:9092")
  .option("subscribe", "promedios")
  .load())

In [29]:
types = tiposStreamingDF.dtypes

In [32]:
types

[('key', 'binary'),
 ('value', 'binary'),
 ('topic', 'string'),
 ('partition', 'int'),
 ('offset', 'bigint'),
 ('timestamp', 'timestamp'),
 ('timestampType', 'int')]

In [33]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import pyspark.sql.functions as F

esquema = StructType([\
  StructField("tipo", StringType()),\
  StructField("total", DoubleType())\
])

parsedDF = tiposStreamingDF\
     .select("value")\
     .withColumn("value", F.col("value").cast(StringType()))\
     .withColumn("parejas", F.from_json(F.col("value"), esquema))\
     .withColumn("tipo", F.col("parejas.tipo"))\
     .withColumn("total", F.col("parejas.total"))


In [35]:
promediosStreamingDF = promediarValores(parsedDF)

salida = promediosStreamingDF\
                    .writeStream\
                    .queryName("AgregacionPromedios")\
                    .outputMode("complete")\
                    .format("memory")\
                    .start()

22/07/25 04:40:55 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2e97f4be-4c95-4849-91d5-8659b42509e3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [36]:
promediosDF = spark.sql("select * from AgregacionPromedios")

In [51]:
promediosDF.show()

+-------+--------+
|   tipo|promedio|
+-------+--------+
|ingreso|    6.75|
|  gasto|     2.5|
+-------+--------+



In [52]:
promediosDF.show()

+-------+--------+
|   tipo|promedio|
+-------+--------+
|ingreso|    6.75|
|  gasto|     2.5|
+-------+--------+



In [ ]:
{"tipo": "gasto", "total": 3.5}
{"tipo": "ingreso", "total": 7.0}
{"tipo": "ingreso", "total": 6.5}
{"tipo": "gasto", "total": 1.5}
{"tipo": "gasto", "total": 2.5}